In [47]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import sys
import iprompt.data
from transformers import AutoTokenizer, OPTForCausalLM, AutoModelForCausalLM
# from iprompt.data import TASKS_GALACTICA
import transformers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
results_dir = '/home/chansingh/iprompt/experiments/results'

In [49]:
rs = {
    fname.replace('.pkl', ''): pkl.load(open(oj(results_dir, fname), 'rb'))
    for fname in os.listdir(results_dir)
    if fname.endswith('.pkl')
    and fname.startswith('uniprot')
}

In [50]:
rs.keys()

dict_keys(['uniprot_cytoplasm_membrane_100_4', 'uniprot_rna-binding_atp-binding_100_5', 'uniprot_cytoplasm_membrane_100_5', 'uniprot_rna-binding_atp-binding_100_3', 'uniprot_rna-binding_atp-binding_100_0', 'uniprot_cytoplasm_membrane_100_1', 'uniprot_rna-binding_atp-binding_100_1', 'uniprot_rna-binding_atp-binding_100_4', 'uniprot_cytoplasm_membrane_100_0', 'uniprot_rna-binding_atp-binding_100_6', 'uniprot_cytoplasm_membrane_100_7', 'uniprot_cytoplasm_membrane_100_3', 'uniprot_cytoplasm_membrane_100_2', 'uniprot_cytoplasm_membrane_100_6', 'uniprot_rna-binding_atp-binding_100_2'])

In [72]:
def cyto(l: list):
    x = ''.join(l).lower()
    return 'cyto' in x or 'membrane' in x

def rna(l: list):
    x = ''.join(l).lower()
    return 'rna' in x or 'atp' in x

def mrr(x: list, f):
    for i, y in enumerate(x):
        if f([y]):
            return 1 / (i + 1)
    return 0

r = {}
for k in rs:
    l = rs[k]['prefixes']
    if 'cyto' in k:
        r[k] = [cyto(l[:20]), cyto(l[:5]), mrr(l, cyto)]
    else:
        r[k] = [rna(l[:20]), rna(l[:5]), mrr(l, rna)]
    
df = pd.DataFrame.from_dict(r)
df.index = ['Top-5 prompt correctness', 'Top-20 prompt correctness', 'MRR']
ks_baseline = [k for k in df.columns if 'baseline' in k]
ks_cyto = [k for k in df.columns if 'cyto' in k]
ks_rna = [k for k in df.columns if not 'cyto' in k]
df_baseline = df[ks_baseline]
# df_baseline_cyto = df[[k for k in ks_baseline if k in ks_cyto]]
# df_baseline_rna = df[[k for k in ks_baseline if k in ks_rna]]
df_cyto = df[[k for k in ks_cyto if k not in ks_baseline]]
df_rna = df[[k for k in ks_rna if k not in ks_baseline]]

In [73]:
scores_cyto = df_cyto.mean(axis=1).round(2).astype(str) + \
    ' $\pm$ ' + \
    (df_cyto.std(axis=1) /
     np.sqrt(df_cyto.shape[1])).round(2).astype(str)
scores_rna = df_rna.mean(axis=1).round(2).astype(str) + \
    ' $\pm$ ' + \
    (df_rna.std(axis=1) /
        np.sqrt(df_rna.shape[1])).round(2).astype(str)
scores_baseline = df_baseline.mean(axis=1)

In [74]:
d = pd.concat((scores_cyto, scores_rna, scores_baseline), axis=1)
d.columns = ['iPrompt cyto-membrane', 'iPrompt binding', 'Null data']
d = d.loc[['MRR', 'Top-5 prompt correctness', 'Top-20 prompt correctness']]
print(d.to_latex(escape=False))

\begin{tabular}{lllr}
\toprule
{} & iPrompt cyto-membrane &  iPrompt binding &  Null data \\
\midrule
MRR                       &       0.13 $\pm$ 0.06 &  0.05 $\pm$ 0.03 &        NaN \\
Top-5 prompt correctness  &       0.75 $\pm$ 0.16 &  0.29 $\pm$ 0.18 &        NaN \\
Top-20 prompt correctness &       0.12 $\pm$ 0.12 &  0.14 $\pm$ 0.14 &        NaN \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3593959/2859349087.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(d.to_latex(escape=False))


In [ ]:
for k in rs:
    print(rs[k]['prefixes'])

# print example

In [125]:
from imodelsx import explain_dataset_iprompt, get_add_two_numbers_dataset
from iprompt.data import TASKS_GALACTICA

In [126]:
task_name = f'tox21_0'

In [128]:
# get task
task = TASKS_GALACTICA[task_name]

# get data
df = task['gen_func']()
input_strings = df['input'].values
output_strings = df['output'].values

/home/chansingh/iprompt/iprompt/data_utils/galactica.py:35: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  control = d.loc[(d.sum(axis=1) == 0) & (d.isna().sum(axis=1) == 0)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:36: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tox = d.loc[(d.sum(axis=1) == 1) & (d.iloc[:, tox_target] == 1)]
/home/chansingh/iprompt/iprompt/data_utils/galactica.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tox'].iloc[n:] = 0

In [134]:
output_strings[0]

' No.\n\n'